# Import

In [ ]:
import pandas as pd
import os
from dotenv import load_dotenv
from utils import *
from pathlib import Path

load_dotenv()

# Constants & Parameters

In [ ]:
LIMIT_PER_REQUEST = 1000
MAX_FOLLOWINGS_REQUESTED = 5000
project_dir = Path(__file__).resolve().parents[1]

# Request Access Token

In [ ]:
print("Request Access Token")
print("-------------------------")
access_token = request_access_token()
access_token_string = "Bearer " + access_token

addToClipBoard(access_token_string)
print("Access Token: ", access_token_string)

# Request Followings

In [ ]:
print("Request All Followings")
print("-------------------------")
# Fetch all followings for the given profile ID
followings = get_all_followings(os.getenv("PHAVER_PROFILE_ID"), LIMIT_PER_REQUEST, MAX_FOLLOWINGS_REQUESTED, access_token)

# Get the points for each following
# followings = get_all_points(followings, access_token)

# Save the followings to followings.json
save_to_json(followings, 'data/followings.json')

print(f"Successfully saved {len(followings)} followings to 'data/followings.json'.")

## Convert JSON to DataFrame
# Flatten the data
followings_df = flatten_followings(followings)

# Add the current datetime to the DataFrame
followings_df['updatedAt'] = pd.Timestamp.now(tz='UTC')

# Add a tempo column to mark unfollowed users
followings_df['tempo_unfollowed'] = False

# Save the DataFrame to a CSV file
followings_df.to_csv(project_dir / 'data/followings.csv', index=False)


# Load Followings Dataframe

In [ ]:
print("\nLoad Followings Dataframe")
print("-------------------------")
followings_df = pd.read_csv(project_dir / 'data/followings.csv')
print(f"Loaded {len(followings_df)} followings.")

# Filter profiles to unfollow

In [ ]:
####################
# Filter profiles to unfollow
####################
print("\nFilter profiles to unfollow")
print("-------------------------")
profiles_to_unfollow_df = followings_df[
    (followings_df['followerCount'] < 3000) & 
    (followings_df['credLevel'] < 3) &
    (followings_df['badge'].isnull()) &
    (followings_df['verification'].isnull()) &
    # (followings_df['lensProfile.isUserFollowing'] != True) &
    (followings_df['tempo_unfollowed'] == False) 
]
print(f"Found {len(profiles_to_unfollow_df)} profiles to unfollow.")

# Unfollow profiles

In [ ]:
error_count = 0
error_limit = 5

try:
    # Unfollow Profiles
    for index, following in profiles_to_unfollow_df.head(120).iterrows():
        try:
            unfollow_user(os.getenv("PHAVER_PROFILE_ID"), following['id'], 'all', access_token)
        except ValueError as e:
            print(f"{e.__class__.__name__} on Profile `{following['username']}` : {e}")
            if error_count >= error_limit:
                break
            else:
                error_count+=1
                continue
        except UnfollowError as e:
            print(f"{e.__class__.__name__} on Profile `{following['username']}` : {e}")
            if e.stopProcess:
                break
            else : 
                if error_count >= error_limit:
                    break
                else:
                    error_count+=1
                    continue
        print(f"Unfollowed {following['username']}")
        followings_df.loc[index, 'tempo_unfollowed'] = True
finally:
    # Save modified following list
    followings_df.to_csv(project_dir / 'data/followings.csv', index=False)
    print("Saved followings list.")